# Part 4.5. RNN Seq2Seq

## 1. Seq2Seq
### 1.1. RNN vs Seq2Seq
**RNN**은 보통 문장을 구성하는 단어가 입력이 되면 그 때마다 출력을 내놓는다. 반면 **Seq2Seq**는 반면 입력이 완료될 때까지 기다린 다음에 출력을 한다.

![21-1.png](./img/21-1.png)

### 1.2. Encoder and Decoder
**Seq2Seq**는 다음과 같이 **Encoder**와 **Decoder**로 구성이 되어 있다.

![21-2.png](./img/21-2.png)

1. **Encoder**에서 들은 모든 단어들의 sequence들을 어떤 vector의 형태로 압축한다.
2. 압축된 vector를 **Decoder**에 전달한다.
3. 이 vector를 **hidden state**에 넣어주고 첫 vector일 경우 **start flag**를 삽입해서 학습을 시작한다.
4. 첫 번째 vector로 인해 나온 출력을 다시 두 번째 셀의 입력으로 넣어 다음 단어를 예측한다.

## 2. Implementation

### 2.1. 모듈 임포트

In [0]:
import random
import torch
import torch.nn as nn
from torch import optim

In [0]:
torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### 2.2. Raw 텍스트

In [0]:
# (영어 문장, 한국어 문장) 을 번역할 예정
raw = ["I feel hungry.	나는 배가 고프다.",
       "Pytorch is very easy.	파이토치는 매우 쉽다.",
       "Pytorch is a framework for deep learning.	파이토치는 딥러닝을 위한 프레임워크이다.",
       "Pytorch is very clear to use.	파이토치는 사용하기 매우 직관적이다."]

### 2.3. 텍스트 전처리

In [0]:
# SOS(Start Of Sentence)와 EOS(End of Sentence)
# Decoder가 문장의 첫과 끝을 인식하기 위한 것
SOS_token = 0
EOS_token = 1

In [0]:
# 단어 딕셔너리
class Vocab:
    def __init__(self):
        self.vocab2index = {"<SOS>": SOS_token, "<EOS>": EOS_token}   # 단어 -> 인덱스
        self.index2vocab = {SOS_token: "<SOS>", EOS_token: "<EOS>"}   # 인덱스 -> 단어
        self.vocab_count = {}                                         # 단어의 개수
        self.n_vocab = len(self.vocab2index)

    # 단어를 추가하는 함수
    def add_vocab(self, sentence):
        for word in sentence.split(" "):
            if word not in self.vocab2index:
                self.vocab2index[word] = self.n_vocab
                self.vocab_count[word] = 1
                self.index2vocab[self.n_vocab] = word
                self.n_vocab += 1
            else:
                self.vocab_count[word] += 1

In [0]:
# Source 데이터와 Target 데이터에서 MAX_LENGTH를 초과하는 문장을 filter하기 위한 함수
def filter_pair(pair, source_max_length, target_max_length):
    return len(pair[0].split(" ")) < source_max_length and len(pair[1].split(" ")) < target_max_length

In [0]:
# Source 텍스트와 Target 텍스트를 전처리하는 함수
def preprocess(corpus, source_max_length, target_max_length):
    print("reading corpus...")

    # 영문장과 국문장을 짝짓기
    pairs = []            
    for line in corpus:
        pairs.append([s for s in line.strip().lower().split("\t")])
    print("Read {} sentence pairs".format(len(pairs)))

    # 짝지은 문장에서 MAX_LENGTH를 초과하는 문장을 필터링
    pairs = [pair for pair in pairs if filter_pair(pair, source_max_length, target_max_length)]
    print("Trimmed to {} sentence pairs".format(len(pairs)))

    # Source 문장과 Target 문장의 Vocab 딕셔너리 생성
    source_vocab = Vocab()
    target_vocab = Vocab()

    print("Counting words...")
    for pair in pairs:
        source_vocab.add_vocab(pair[0])
        target_vocab.add_vocab(pair[1])
    print("source vocab size =", source_vocab.n_vocab)
    print("target vocab size =", target_vocab.n_vocab)

    return pairs, source_vocab, target_vocab

In [0]:
# 문장의 최대 길이 지정
SOURCE_MAX_LENGTH = 10
TARGET_MAX_LENGTH = 12

In [19]:
# 각 문장을 전처리
load_pairs, load_source_vocab, load_target_vocab = preprocess(raw, SOURCE_MAX_LENGTH, TARGET_MAX_LENGTH)
print(random.choice(load_pairs))

reading corpus...
Read 4 sentence pairs
Trimmed to 4 sentence pairs
Counting words...
source vocab size = 17
target vocab size = 13
['pytorch is a framework for deep learning.', '파이토치는 딥러닝을 위한 프레임워크이다.']


### 2.4. Encoder 및 Decoder 정의

In [0]:
# Encoder 정의
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    # embedding -> gru
    # embedding은 입력으로 들어온 희소행렬을 더 작은 행렬로 변환
    def forward(self, x, hidden):
        x = self.embedding(x).view(1, 1, -1)
        x, hidden = self.gru(x, hidden)
        return x, hidden

In [0]:
# Decoder 정의
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        # index -> vocab로 바꿔주기 위한 layer
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    # embedding -> gru -> softmax
    def forward(self, x, hidden):
        x = self.embedding(x).view(1, 1, -1)
        x, hidden = self.gru(x, hidden)
        x = self.softmax(self.out(x[0]))
        return x, hidden

In [0]:
# Encoder와 Decoder 생성
enc_hidden_size = 16
dec_hidden_size = enc_hidden_size
enc = Encoder(load_source_vocab.n_vocab, enc_hidden_size).to(device)
dec = Decoder(dec_hidden_size, load_target_vocab.n_vocab).to(device)

### 2.5. 학습

In [0]:
# 문장 one-hot encoding 및 tensor로 변환
def tensorize(vocab, sentence):
    indexes = [vocab.vocab2index[word] for word in sentence.split(" ")]
    indexes.append(vocab.vocab2index["<EOS>"])
    return torch.Tensor(indexes).long().to(device).view(-1, 1)

In [0]:
# Seq2Seq를 학습하는 함수
def train(pairs, source_vocab, target_vocab, encoder, decoder, n_iter, print_every=1000, learning_rate=0.01):
    loss_total = 0

    # encoder와 decoder의 optimizer 설정
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

    # source 데이터와 target 데이터로 나눔
    training_batch = [random.choice(pairs) for _ in range(n_iter)]
    training_source = [tensorize(source_vocab, pair[0]) for pair in training_batch]
    training_target = [tensorize(target_vocab, pair[1]) for pair in training_batch]

    # 손실함수 설정 : Negative Log Likelihood
    criterion = nn.NLLLoss()

    for i in range(1, n_iter + 1):
        source_tensor = training_source[i - 1]
        target_tensor = training_target[i - 1]

        encoder_hidden = torch.zeros([1, 1, encoder.hidden_size]).to(device)

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        source_length = source_tensor.size(0)
        target_length = target_tensor.size(0)

        loss = 0

        for enc_input in range(source_length):
            _, encoder_hidden = encoder(source_tensor[enc_input], encoder_hidden)

        # Decoder의 입력에 SOS 토큰을 삽입
        decoder_input = torch.Tensor([[SOS_token]]).long().to(device)
        # Encoder의 출력을 Decoder의 입력으로 넣어줌
        decoder_hidden = encoder_hidden 

        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            # 방법 1. 출력으로 넣은 걸 입력으로 다시 넣기
            # 방법 2. teacher forcing : 실제 데이터를 넣어주는 기법
            # 어느 게 더 좋은 건 아니고 적절히 해줌. 한 30, 40% 정도로 설정
            decoder_input = target_tensor[di] 

        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        loss_iter = loss.item() / target_length
        loss_total += loss_iter

        if i % print_every == 0:
            loss_avg = loss_total / print_every
            loss_total = 0
            print("[{} - {}%] loss = {:05.4f}".format(i, i / n_iter * 100, loss_avg))

In [26]:
# Seq2Seq 모델 학습
train(load_pairs, load_source_vocab, load_target_vocab, enc, dec, 5000, print_every=1000)

[1000 - 20.0%] loss = 0.7299
[2000 - 40.0%] loss = 0.1562
[3000 - 60.0%] loss = 0.0478
[4000 - 80.0%] loss = 0.0220
[5000 - 100.0%] loss = 0.0140


### 2.6. 모델 성능 체크

In [0]:
# 모델 성능을 평가하기 위한 함수
def evaluate(pairs, source_vocab, target_vocab, encoder, decoder, target_max_length):
    for pair in pairs:
        print(">", pair[0])
        print("=", pair[1])
        source_tensor = tensorize(source_vocab, pair[0])
        source_length = source_tensor.size()[0]
        encoder_hidden = torch.zeros([1, 1, encoder.hidden_size]).to(device)

        for ei in range(source_length):
            _, encoder_hidden = encoder(source_tensor[ei], encoder_hidden)

        decoder_input = torch.Tensor([[SOS_token]]).long().to(device)
        decoder_hidden = encoder_hidden
        decoded_words = []

        for di in range(target_max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            _, top_index = decoder_output.data.topk(1)
            if top_index.item() == EOS_token:
                decoded_words.append("<EOS>")
                break
            else:
                decoded_words.append(target_vocab.index2vocab[top_index.item()])

            decoder_input = top_index.squeeze().detach()

        predict_words = decoded_words
        predict_sentence = " ".join(predict_words)
        print("<", predict_sentence)
        print("")

In [30]:
# 모델 성능 체크
evaluate(load_pairs, load_source_vocab, load_target_vocab, enc, dec, TARGET_MAX_LENGTH)

> i feel hungry.
= 나는 배가 고프다.
< 나는 배가 고프다. <EOS>

> pytorch is very easy.
= 파이토치는 매우 쉽다.
< 파이토치는 매우 쉽다. <EOS>

> pytorch is a framework for deep learning.
= 파이토치는 딥러닝을 위한 프레임워크이다.
< 파이토치는 딥러닝을 위한 프레임워크이다. <EOS>

> pytorch is very clear to use.
= 파이토치는 사용하기 매우 직관적이다.
< 파이토치는 사용하기 매우 직관적이다. <EOS>

